In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
data = {
    'Date': pd.date_range(start='1/1/2010', periods=120, freq='M'),
    'MilkProduction': np.random.randint(1000, 2000, size=120)
}
df = pd.DataFrame(data)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['MilkProduction'].values.reshape(-1, 1))

In [ ]:
sequence_length = 12  # Consideramos los últimos 12 meses para predecir el siguiente
X = []
y = []
for i in range(len(scaled_data) - sequence_length):
    X.append(scaled_data[i:i+sequence_length])
    y.append(scaled_data[i+sequence_length])
X = np.array(X)
y = np.array(y)

In [ ]:
split_ratio = 0.8
split_index = int(split_ratio * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [ ]:
model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

In [ ]:
#grafica con la perdida
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.show()

In [ ]:
test_loss = model.evaluate(X_test, y_test)
print('Test loss:', test_loss)

In [ ]:
#predicciones proximo año
future_months = 12
future_predictions = []
last_sequence = X[-1]
for i in range(future_months):
    prediction = model.predict(last_sequence.reshape(1, sequence_length, 1))
    future_predictions.append(prediction[0][0])
    last_sequence = np.roll(last_sequence, -1)
    last_sequence[-1] = prediction[0][0]

In [ ]:
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

In [ ]:
future_dates = pd.date_range(start=df['Date'].iloc[-1] + pd.DateOffset(months=1), periods=future_months, freq='M')
future_df = pd.DataFrame({'Date': future_dates, 'MilkProduction': future_predictions.flatten()})
print(future_df)